In [4]:
# R-10.1

def pop(self, last = True):

  if last:
    temp = self[-1]
    del self[-1]
  else:
    temp = self[last]
    del self[last]
  
  return temp

# 1. MutableMapping has 5 abstract method: __getitem__, __setitem__, __delitem__, __iter__, __len__. In pop() we oonly need __getitem__ and __delitem__. note __delitem__ won't return the item

In [5]:
# R-10.2

def items(self):
  result = []
  for key in iter(self):
    value = self[key]
    result.append(key, value)
  return result

# if applied directly into the UnsortedTableMap, iter() only gives you the key, and need call self[key] to find value, both of them are O(n)
# so totally it is O(n^2)


# 1. items() return a set-like view of (k, v) tuple for all entries, which is not a iterator

In [6]:
# R-10.3

from collections import MutableMapping

class MapBase(MutableMapping):
  class _Item:
    __slots__ = '_key', '_value'
    
    def __init__(self, k, v):
      self._key = k
      self._value = v

    def __eq__(self, other):
      return self._key == other._key

    def __ne__(self, other):
      return not (self == other)

    def __It__(self, other):
      return self._key < other._key

class UnsortedTableMap(MapBase):
  

  def __init__(self):
    self._table = []


  def __getitem__(self, k):
    for item in self._table:
      if k == item._key:
        return item._value
    raise KeyError('Key Error: ' + repr(k))


  def __setitem__(self, k, v):
    #  d[k] = v
    for item in self._table:
      if k == item._key:
        item._value = v
        return

    self._table.append(self._Item(k, v))


  def __delitem__(self, k):
    for j in range(len(self._table)):
      if k == self._table[j]._key:
        self._table.pop(j)
        return
    raise KeyError('Key Error: ' + repr(k))


  def __len__(self):
    return len(self._table)


  def __iter__(self):
    for item in self._table:
      yield item._key

  def items(self):
    # return a list of tuple
    result = []
    # we don't use iter() at here because it can only return key, but not value
    for item in self._table:
      result.append((item._key, item._value))
    return result
    # for item in iter(self):
    #   result.append(item)
    # return result

if __name__ == '__main__':
  example = UnsortedTableMap()
  s = [('a', 1), ('b', 2), ('c', 3), ('d', 4)]
  # s = {'a': 1, 'b': 2, 'c': 3, 'd': 4}

  s = dict(s)
  for key, value in s.items():
    print()
    example[key] = value

  print(example.items())

  del example['c']

  for i in iter(example):
    print(i)

#  1. we don't use iter() in the items() method because it can only return key, but not value
#  2. dictionary don't accept member function append(), you need to use d[key] = value to add new items, and for key in d.keys(); for value in d.values(); for key, value in d.items() to traverse a dictionary
#  3. s = [('a', 1), ('b', 2), ('c', 3), ('d', 4)], s is a list with key-value pair tuples, you can use for k, v in s:
#     but s = {'a': 1, 'b': 2, 'c': 3, 'd': 4}, s is a dictionary, you must use for k, v in s.items():
#     dict(s), s is still a list; s = dict(s), s is converted to a dictionary





[('a', 1), ('b', 2), ('c', 3), ('d', 4)]
a
b
d


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
# R-10.4
'''
all of the keys are different, so all the called __setitem__ method do a full loop before add this pair 


'''

'\nall of the keys are different, so all the called __setitem__ method do a full loop before add this pair \n\n\n'

In [8]:
# R-10.5

class _DoublyLinkedBase:
  """A base class providing a doubly linked list representation."""

  #-------------------------- nested _Node class --------------------------
  # nested _Node class
  class _Node:
    """Lightweight, nonpublic class for storing a doubly linked node."""
    __slots__ = '_element', '_prev', '_next'            # streamline memory

    def __init__(self, element, prev, next):            # initialize node's fields
      self._element = element                           # user's element
      self._prev = prev                                 # previous node reference
      self._next = next                                 # next node reference

  #-------------------------- list constructor --------------------------

  def __init__(self):
    """Create an empty list."""
    self._header = self._Node(None, None, None)
    self._trailer = self._Node(None, None, None)
    self._header._next = self._trailer                  # trailer is after header
    self._trailer._prev = self._header                  # header is before trailer
    self._size = 0                                      # number of elements

  #-------------------------- public accessors --------------------------

  def __len__(self):
    """Return the number of elements in the list."""
    return self._size

  def is_empty(self):
    """Return True if list is empty."""
    return self._size == 0

  #-------------------------- nonpublic utilities --------------------------

  def _insert_between(self, e, predecessor, successor):
    """Add element e between two existing nodes and return new node."""
    newest = self._Node(e, predecessor, successor)      # linked to neighbors
    predecessor._next = newest
    successor._prev = newest
    self._size += 1
    return newest

  def _delete_node(self, node):
    """Delete nonsentinel node from the list and return its element."""
    predecessor = node._prev
    successor = node._next
    predecessor._next = successor
    successor._prev = predecessor
    self._size -= 1
    element = node._element                             # record deleted element
    node._prev = node._next = node._element = None      # deprecate node
    return element                                      # return deleted element


class PositionalList(_DoublyLinkedBase):
  """A sequential container of elements allowing positional access."""

  #-------------------------- nested Position class --------------------------
  class Position:
    """An abstraction representing the location of a single element.

    Note that two position instaces may represent the same inherent
    location in the list.  Therefore, users should always rely on
    syntax 'p == q' rather than 'p is q' when testing equivalence of
    positions.
    """

    def __init__(self, container, node):
      """Constructor should not be invoked by user."""
      self._container = container
      self._node = node
    
    def element(self):
      """Return the element stored at this Position."""
      return self._node._element
      
    def __eq__(self, other):
      """Return True if other is a Position representing the same location."""
      return type(other) is type(self) and other._node is self._node

    def __ne__(self, other):
      """Return True if other does not represent the same location."""
      return not (self == other)               # opposite of __eq__
    
  #------------------------------- utility methods -------------------------------
  def _validate(self, p):
    """Return position's node, or raise appropriate error if invalid."""
    if not isinstance(p, self.Position):
      raise TypeError('p must be proper Position type')
    if p._container is not self:
      raise ValueError('p does not belong to this container')
    if p._node._next is None:                  # convention for deprecated nodes
      raise ValueError('p is no longer valid')
    return p._node

  def _make_position(self, node):
    """Return Position instance for given node (or None if sentinel)."""
    if node is self._header or node is self._trailer:
      return None                              # boundary violation
    else:
      return self.Position(self, node)         # legitimate position
    
  #------------------------------- accessors -------------------------------
  def first(self):
    """Return the first Position in the list (or None if list is empty)."""
    return self._make_position(self._header._next)

  def last(self):
    """Return the last Position in the list (or None if list is empty)."""
    return self._make_position(self._trailer._prev)

  def before(self, p):
    """Return the Position just before Position p (or None if p is first)."""
    node = self._validate(p)
    return self._make_position(node._prev)

  def after(self, p):
    """Return the Position just after Position p (or None if p is last)."""
    node = self._validate(p)
    return self._make_position(node._next)

  def __iter__(self):
    """Generate a forward iteration of the elements of the list."""
    cursor = self.first()
    while cursor is not None:
      yield cursor.element()
      cursor = self.after(cursor)

  #------------------------------- mutators -------------------------------
  # override inherited version to return Position, rather than Node
  def _insert_between(self, e, predecessor, successor):
    """Add element between existing nodes and return new Position."""
    node = super()._insert_between(e, predecessor, successor)
    return self._make_position(node)

  def add_first(self, e):
    """Insert element e at the front of the list and return new Position."""
    return self._insert_between(e, self._header, self._header._next)

  def add_last(self, e):
    """Insert element e at the back of the list and return new Position."""
    return self._insert_between(e, self._trailer._prev, self._trailer)

  def add_before(self, p, e):
    """Insert element e into list before Position p and return new Position."""
    original = self._validate(p)
    return self._insert_between(e, original._prev, original)

  def add_after(self, p, e):
    """Insert element e into list after Position p and return new Position."""
    original = self._validate(p)
    return self._insert_between(e, original, original._next)

  def delete(self, p):
    """Remove and return the element at Position p."""
    original = self._validate(p)
    return self._delete_node(original)  # inherited method returns element

  def replace(self, p, e):
    """Replace the element at Position p with e.

    Return the element formerly at Position p.
    """
    original = self._validate(p)
    old_value = original._element       # temporarily store old element
    original._element = e               # replace with new element
    return old_value                    # return the old element value


class LinkedUnsortedTableMap(MapBase):
  

  def __init__(self):
    self._table = PositionalList()


  def __getitem__(self, k):
    for item in self._table:
      if k == item._key:
        return item._value
    raise KeyError('Key Error: ' + repr(k))


  def __setitem__(self, k, v):
    #  d[k] = v
    for item in self._table:
      if k == item._key:
        item._value = v
        return

    self._table.add_last(self._Item(k, v))


  def __delitem__(self, k):
    p = self._table.first()
    for item in iter(self._table):
      if k == item._key:
        self._table.delete(p)
        return
      p = self._table.after(p)
    raise KeyError('Key Error: ' + repr(k))


  def __len__(self):
    return len(self._table)


  def __iter__(self):
    for item in self._table:
      yield item._key

  def items(self):
    # return a list of tuple
    result = []
    # we don't use iter() at here because it can only return key, but not value
    for item in self._table:
      result.append((item._key, item._value))
    return result
    # for item in iter(self):
    #   result.append(item)
    # return result


if __name__ == '__main__':
  example = LinkedUnsortedTableMap()
  s = [('a', 1), ('b', 2), ('c', 3), ('d', 4)]
  # s = {'a': 1, 'b': 2, 'c': 3, 'd': 4}

  s = dict(s)
  for key, value in s.items():
    print()
    example[key] = value

  print(example.items())

  for i in iter(example):
    print(i)

  del example['c']

  for i in iter(example):
    print(i)

  print(len(example))

#  1. the difficult is in delete method, when we want to connect the key-value pair with the position in the PositionalList, an easy way is to use a variable p to record the current position in the for loop
#     just like the cursor in PositionalList.__iter__





[('a', 1), ('b', 2), ('c', 3), ('d', 4)]
a
b
c
d
a
b
d
3


In [9]:
# R-10.6
'''
separate chaining can tolerate a load factor higher than 1. While it is not efficient, the bucket is a container and can accept more than 1 items
open addressing, including linear probing and its variant cannot tolerate a load factor higher than 1, as an item is stored directly in the cell of the bucket array


'''

'\nseparate chaining can tolerate a load factor higher than 1. While it is not efficient, the bucket is a container and can accept more than 1 items\nopen addressing, including linear probing and its variant cannot tolerate a load factor higher than 1, as an item is stored directly in the cell of the bucket array\n\n\n'

In [10]:
# R-10.7

def __hash__(self):
  # position itself is a class type, but it always has a nonpublic variable node, which is an integer
  return hash(self.Position._node)

In [11]:
# R-10.8

#  consider it as a character string and use cyclic-shift hash code (5-bit shift), which can take position into account


In [12]:
# R-10.9
'''
the hash code for integer is itself

k   h(k)
12  8
44  5
13  0
88  5
23  8
94  1
11  5
39  1
20  10
16  9
5   9


  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|  10
13   94           44        12  16   20
    39           88        23   5
                11

'''

'\nthe hash code for integer is itself\n\nk   h(k)\n12  8\n44  5\n13  0\n88  5\n23  8\n94  1\n11  5\n39  1\n20  10\n16  9\n5   9\n\n\n  0|  1|  2|  3|  4|  5|  6|  7|  8|  9|  10\n13   94           44        12  16   20\n    39           88        23   5\n                11\n\n'

In [13]:
# R-10.10
'''
the number of item n = 11
the sizr of bucket array N = 11

k  h(k)|j  
12  8     
44  5
13  0
88  5, occupied; (j+1) mod N = 6 mod 11 = 6
23  8, occupied; (j+1) mod N = 9 mod 11 = 9
94  1
11  5, occupied; (j+1) mod N = 6 mod 11 = 6, occupied; (j+2) mod N = 7 mod 11 = 7
39  1, occupied; (j+1) mod N = 2 mod 11 = 2
20  10
16  9, occupied; (j+1) mod N = 10 mod 11 = 10, occupied; (j+2) mod N = 11 mod 11 = 0, occupied; (j+3) mod N = 12 mod 11 = 1, occupied; (j+4) mod N = 13 mod 11 = 2, occupied;
           (j+5) mod N = 14 mod 11 = 3
5   9, occupied; (j+1) mod N = 10 mod 11 = 10, occupied; (j+2) mod N = 11 mod 11 = 0, occupied; (j+3) mod N = 12 mod 11 = 1, occupied; (j+4) mod N = 13 mod 11 = 2, occupied;
           (j+5) mod N = 14 mod 11 = 3, occupied; (j+6) mod N = 15 mod 11 = 4




'''

'\nthe number of item n = 11\nthe sizr of bucket array N = 11\n\nk  h(k)|j  \n12  8     \n44  5\n13  0\n88  5, occupied; (j+1) mod N = 6 mod 11 = 6\n23  8, occupied; (j+1) mod N = 9 mod 11 = 9\n94  1\n11  5, occupied; (j+1) mod N = 6 mod 11 = 6, occupied; (j+2) mod N = 7 mod 11 = 7\n39  1, occupied; (j+1) mod N = 2 mod 11 = 2\n20  10\n16  9, occupied; (j+1) mod N = 10 mod 11 = 10, occupied; (j+2) mod N = 11 mod 11 = 0, occupied; (j+3) mod N = 12 mod 11 = 1, occupied; (j+4) mod N = 13 mod 11 = 2, occupied;\n           (j+5) mod N = 14 mod 11 = 3\n5   9, occupied; (j+1) mod N = 10 mod 11 = 10, occupied; (j+2) mod N = 11 mod 11 = 0, occupied; (j+3) mod N = 12 mod 11 = 1, occupied; (j+4) mod N = 13 mod 11 = 2, occupied;\n           (j+5) mod N = 14 mod 11 = 3, occupied; (j+6) mod N = 15 mod 11 = 4\n\n\n\n\n'

In [14]:
# R-10.11
'''
k  h(k)|j
12  8   (h(k) + f(0)) mod N = 8 mod 11 = 8
44  5   (h(k) + f(0)) mod N = 5 mod 11 = 5
13  0   (h(k) + f(0)) mod N = 0 mod 11 = 0
88  5   (h(k) + f(0)) mod N = 5 mod 11 = 5, occupied; (h(k) + f(1)) mod N = 6 mod 11 = 6
23  8   (h(k) + f(0)) mod N = 8 mod 11 = 8, occupied; (h(k) + f(1)) mod N = 9 mod 11 = 9
94  1   (h(k) + f(0)) mod N = 1 mod 11 = 1
11  5   (h(k) + f(0)) mod N = 5 mod 11 = 5, occupied; (h(k) + f(1)) mod N = 6 mod 11 = 6, occupied; (h(k) + f(2)) mod N = 9 mod 11 = 9, occupied; (h(k) + f(3)) mod N = 14 mod 11 = 3
39  1   (h(k) + f(0)) mod N = 1 mod 11 = 1, occupied; (h(k) + f(1)) mod N = 2 mod 11 = 2
20  10  (h(k) + f(0)) mod N = 10 mod 11 = 10,
16  9   (h(k) + f(0)) mod N = 9 mod 11 = 9, occupied; (h(k) + f(1)) mod N = 10 mod 11 = 10, occupied; (h(k) + f(2)) mod N = 13 mod 11 = 2, occupied; (h(k) + f(3)) mod N = 18 mod 11 = 7
5   9  the quadratic probing method fails



'''

n = 10000000
for i in range(0, n):
  if (9 + pow(i, 2)) % 11 == 4:
    print(i)

In [15]:
# R-10.12
'''
k  h(k)      h'(k)         
12  8   7 - (12 mod 7) = 2    (h(k) + 0*h'(k)) mod N = 8 mod 11 = 8
44  5   7 - (44 mod 7) = 5    (h(k) + 0*h'(k)) mod N = 5 mod 11 = 5
13  0   7 - (13 mod 7) = 1    (h(k) + 0*h'(k)) mod N = 0 mod 11 = 0
88  5   7 - (88 mod 7) = 3    (h(k) + 0*h'(k)) mod N = 5 mod 11 = 5, occupied; (h(k) + 1*h'(k)) mod N = 8 mod 11 = 8, occupied; (h(k) + 2*h'(k)) mod N = 11 mod 11 = 0, occupied;
                      (h(k) + 3*h'(k)) mod N = 14 mod 11 = 3
23  8   7 - (23 mod 7) = 5    (h(k) + 0*h'(k)) mod N = 8 mod 11 = 8, occupied; (h(k) + 1*h'(k)) mod N = 13 mod 11 = 2
94  1   7 - (94 mod 7) = 4    (h(k) + 0*h'(k)) mod N = 1 mod 11 = 1
11  5   7 - (11 mod 7) = 3    (h(k) + 0*h'(k)) mod N = 5 mod 11 = 5, occupied; (h(k) + 1*h'(k)) mod N = 8 mod 11 = 8, occupied; (h(k) + 2*h'(k)) mod N = 11 mod 11 = 0, occupied;
                      (h(k) + 3*h'(k)) mod N = 14 mod 11 = 3, occupied; (h(k) + 4*h'(k)) mod N = 17 mod 11 = 6
39  1   7 - (39 mod 7) = 3    (h(k) + 0*h'(k)) mod N = 1 mod 11 = 1, occupied; (h(k) + 1*h'(k)) mod N = 4 mod 11 = 4
20  10   7 - (20 mod 7) = 1   (h(k) + 0*h'(k)) mod N = 10 mod 11 = 10
16  9   7 - (16 mod 7) = 5    (h(k) + 0*h'(k)) mod N = 9 mod 11 = 9
5   9   7 - (5 mod 7) = 2    (h(k) + 0*h'(k)) mod N = 9 mod 11 = 9, occupied; (h(k) + 1*h'(k)) mod N = 11 mod 11 = 0, occupied; (h(k) + 2*h'(k)) mod N = 13 mod 11 = 2, occupied;
                      (h(k) + 3*h'(k)) mod N = 15 mod 11 = 4, occupied; (h(k) + 4*h'(k)) mod N = 17 mod 11 = 6, occupied; (h(k) + 5*h'(k)) mod N = 19 mod 11 = 8, occupied;
                      (h(k) + 6*h'(k)) mod N = 21 mod 11 = 10, occupied; (h(k) + 7*h'(k)) mod N = 23 mod 11 = 1, occupied; (h(k) + 8*h'(k)) mod N = 25 mod 11 = 3, occupied; 
                      (h(k) + 9*h'(k)) mod N = 27 mod 11 = 5, occupied; (h(k) + 4*h'(k)) mod N = 29 mod 11 = 7



'''



"\nk  h(k)      h'(k)         \n12  8   7 - (12 mod 7) = 2    (h(k) + 0*h'(k)) mod N = 8 mod 11 = 8\n44  5   7 - (44 mod 7) = 5    (h(k) + 0*h'(k)) mod N = 5 mod 11 = 5\n13  0   7 - (13 mod 7) = 1    (h(k) + 0*h'(k)) mod N = 0 mod 11 = 0\n88  5   7 - (88 mod 7) = 3    (h(k) + 0*h'(k)) mod N = 5 mod 11 = 5, occupied; (h(k) + 1*h'(k)) mod N = 8 mod 11 = 8, occupied; (h(k) + 2*h'(k)) mod N = 11 mod 11 = 0, occupied;\n                      (h(k) + 3*h'(k)) mod N = 14 mod 11 = 3\n23  8   7 - (23 mod 7) = 5    (h(k) + 0*h'(k)) mod N = 8 mod 11 = 8, occupied; (h(k) + 1*h'(k)) mod N = 13 mod 11 = 2\n94  1   7 - (94 mod 7) = 4    (h(k) + 0*h'(k)) mod N = 1 mod 11 = 1\n11  5   7 - (11 mod 7) = 3    (h(k) + 0*h'(k)) mod N = 5 mod 11 = 5, occupied; (h(k) + 1*h'(k)) mod N = 8 mod 11 = 8, occupied; (h(k) + 2*h'(k)) mod N = 11 mod 11 = 0, occupied;\n                      (h(k) + 3*h'(k)) mod N = 14 mod 11 = 3, occupied; (h(k) + 4*h'(k)) mod N = 17 mod 11 = 6\n39  1   7 - (39 mod 7) = 3    (h(k) + 0*h

In [16]:
# R-10.13
'''
n entries, initially empty hash table, separate chaining
the worst-case: the compression function map every hash code into one bucket, which is O(1+2+...+n) = O(n^2)
the best-cause: the hash table is large enough and the compression function is good enough, there is no collision, O(1+1+...+1) = O(n)
'''

'\nn entries, initially empty hash table, separate chaining\nthe worst-case: the compression function map every hash code into one bucket, which is O(1+2+...+n) = O(n^2)\nthe best-cause: the hash table is large enough and the compression function is good enough, there is no collision, O(1+1+...+1) = O(n)\n'

In [17]:
# R-10.14
'''
0 | 1| 2| 3| 4| 5| 6| 7| 8| 9| 10| 11| 12| 13| 14| 15| 16| 17| 18
    12 18 41   36 25   54       38 10    90  28        

k   h(k)
54  162 mod 17 = 9
28  84 mod 17 = 16
41  123 mod 17 = 4
18  54 mod 17 = 3
10  30 mod 17 = 13
36  108 mod 17 = 6
25  75 mod 17 = 7
38  114 mod 17 = 12
12  36 mod 17 = 2
90  270 mod 17 = 15



'''

'\n0 | 1| 2| 3| 4| 5| 6| 7| 8| 9| 10| 11| 12| 13| 14| 15| 16| 17| 18\n    12 18 41   36 25   54       38 10    90  28        \n\nk   h(k)\n54  162 mod 17 = 9\n28  84 mod 17 = 16\n41  123 mod 17 = 4\n18  54 mod 17 = 3\n10  30 mod 17 = 13\n36  108 mod 17 = 6\n25  75 mod 17 = 7\n38  114 mod 17 = 12\n12  36 mod 17 = 2\n90  270 mod 17 = 15\n\n\n\n'

In [18]:
# R-10.15

from collections import MutableMapping

class MapBase(MutableMapping):
  class _Item:
    __slots__ = '_key', '_value'
    
    def __init__(self, k, v):
      self._key = k
      self._value = v

    def __eq__(self, other):
      return self._key == other._key

    def __ne__(self, other):
      return not (self == other)

    def __It__(self, other):
      return self._key < other._key

from abc import abstractmethod, ABCMeta
from random import randrange 

class HashMapBase(MapBase, metaclass=ABCMeta):

  def __init__(self, cap=11, p=109345121, f = 0.5):
    self._table = cap * [None]
    self._n = 0
    self._prime = p
    self._scale = randrange(1, p)
    self._shift = randrange(p)
    # load factor, 0.5 by default
    self._f = f

  def _hash_function(self, k):
    return (hash(k) * self._scale + self._shift) % self._prime % len(self._table)

  def __len__(self):
    return self._n

  def __getitem__(self, k):
    j = self._hash_function(k)
    return self._bucket_getitem(j, k)

  def __getitem__(self, k):
    j = self._hash_function(k)
    return self._bucket_getitem(j, k)

  def __setitem__(self, k, v):
    j = self._hash_function(k)
    self._bucket_setitem(j, k, v)
    if self._n > len(self._table) * self._f:
      self._resize(2 * len(self._table) - 1)

  def __delitem__(self, k):
    j = self._hash_function(k)
    self._bucket_delitem(j, k)
    self._n -= 1

  def _resize(self, c):
    old = list(self.item())
    self._table = c * [None]
    self._n = 0
    for (k, v) in old:
      self[k] = v




  # ------abstract method-------
  def _bucket_getitem(self, j, k):
    raise Exception ('_bucket_getitem must be implemented by the subclass!')

  @abstractmethod
  def _bucket_setitem(self, j, k, v):
    '''pass'''

  @abstractmethod
  def _bucket_delitem(self, j, k):
    pass

  @abstractmethod
  def __iter__(self):
    pass





In [19]:
# R-10.16
def _find_slot(self, j, k):
  # the difference between linear probing and quadratic probing is how we update the hash code j
  i = 0
  firstAvail = None
  while True:
    if self._is_available(j):
      if firstAvail is None:
        firstAvail = j
      if self._table[j] is None:
        return False, firstAvail
    elif k == self._table[j]._key:
      return True, j
    i += 1
    # in linear probing we use j + 1
    j = (j + i ** 2) % len(self._table)

In [20]:
# R-10.17
class ProbeHashMap(HashMapBase):
  # use quadratic probing
  _AVAIL = object()

  def _is_available(self, j):
    return self._table[j] is None or self._table[j] is ProbeHashMap._AVAIL

  def _find_slot(self, j, k):
    # the difference between linear probing and quadratic probing is how we update the hash code j
    i = 0
    firstAvail = None
    while True:
      if self._is_available(j):
        if firstAvail is None:
          firstAvail = j
        if self._table[j] is None:
          return False, firstAvail
      elif k == self._table[j]._key:
        return True, j
      i += 1
      # in linear probing we use j + 1
      j = (j + i ** 2) % len(self._table)

  def _bucket_getitem(self, j, k):
    found, s = self._find_slot(j, k)
    if not found:
      raise KeyError('Key Error: ' + repr(k))
    return self._table[s]._value

  def _bucket_setitem(self, j, k, v):
    found, s = self._find_slot(j, k)
    if not found:
      self._table[s] = self._Item(k, v)
      self._n += 1
    else: 
      self._table[s]._value = v

  def _bucket_delitem(self, j, k):
    found, s = self._find_slot(j, k)
    if not found:
      raise KeyError('Key Error: ' + repr(k))
    self._table[s] = ProbeHashMap._AVAIL

  def __iter__(self):
    for j in range(len(self._table)):
      if not self._is_available(j):
        yield self._table[j]._key

In [21]:
# R-10.18
'''
key-value pairs are stored according to related hash code in hash table, which means the key is not stored consecutively



'''

'\nkey-value pairs are stored according to related hash code in hash table, which means the key is not stored consecutively\n\n\n\n'

In [22]:
# R-10.19
'''
when use a sorted list implemented as a doubly linked list, binary search can no loner be used. but we can search the item we want from the begining in the nonpublic _find_index method
use insertion-sort algorithm to add new key-value pair

'''


'\nwhen use a sorted list implemented as a doubly linked list, binary search can no loner be used. but we can search the item we want from the begining in the nonpublic _find_index method\nuse insertion-sort algorithm to add new key-value pair\n\n'

In [23]:
# R-10.20
'''
binary search is O(logn)
O(log(2n) + log(2n-1) + ... + log(n+1)) = O(nlogn)


'''

'\nbinary search is O(logn)\nO(log(2n) + log(2n-1) + ... + log(n+1)) = O(nlogn)\n\n\n'

In [24]:
# R-10.21
'''
the new verson is wrong

| 0| 1| 2| 3| 4|

low = 0, high = 4, mid = 2
assume data[mid]._key == k
return self._fing_index(k, 0, 1), which can on longer find the result


'''

'\nthe new verson is wrong\n\n| 0| 1| 2| 3| 4|\n\nlow = 0, high = 4, mid = 2\nassume data[mid]._key == k\nreturn self._fing_index(k, 0, 1), which can on longer find the result\n\n\n'

In [25]:
# R-10.22
'''
when each pair has lower cost and performance, all the pairs will be inserted at the end. consider of the shifting, the running time is O(1+2+...+n) = O(n^2)
when each pair has a loer cost and higher performance, the running time is still O(n^2), but at the end only the pair with the lowest cost and highest performance left


'''


'\nwhen each pair has lower cost and performance, all the pairs will be inserted at the end. consider of the shifting, the running time is O(1+2+...+n) = O(n^2)\nwhen each pair has a loer cost and higher performance, the running time is still O(n^2), but at the end only the pair with the lowest cost and highest performance left\n\n\n'

In [26]:
# R-10.23
'''



S5  -00                     +00
S4  -00   17                 +00
S3  -00   17         42       +00
S2  -00   17     31   42       +00
S1  -00 12 17     31   42 44     +00
S0  -00 12 17 20 24 31 39 42 44 48 50 +00








'''
import random
random.randint(0,1)

0

In [27]:
# R-10.24
'''
Algorithm SkipDel(k):
  INPUT: Key k
  OUTPUT: The deleted key-value pair

  # use a connectAfter(p) function, that connect position p directly with next(next(p))

  p = SkipSearch(k)
  pre = prev(p)
  v = value(p)
  i = -1
  while i < h:
    i = i+1
    connectAfter(pre)
    while above(pre) is None do
      pre = prev(pre)
    pre = above(pre)
  n = n-1
  return k, v

Algorithm SkipDel(k):
  # the second method 
  INPUT: Key k
  OUTPUT: The deleted key-value pair

  # use a connectBeforeAfter(p) function, that connect positions before and after p directly

  p = SkipSearch(k)
  v = value(p)
  i = -1
  while i < h:
    i = i+1
    connectBeforeAfter(p)
    p = above(p)
  n = n-1
  return k, v




'''

'\nAlgorithm SkipDel(k):\n  INPUT: Key k\n  OUTPUT: The deleted key-value pair\n\n  # use a connectAfter(p) function, that connect position p directly with next(next(p))\n\n  p = SkipSearch(k)\n  pre = prev(p)\n  v = value(p)\n  i = -1\n  while i < h:\n    i = i+1\n    connectAfter(pre)\n    while above(pre) is None do\n      pre = prev(pre)\n    pre = above(pre)\n  n = n-1\n  return k, v\n\nAlgorithm SkipDel(k):\n  # the second method \n  INPUT: Key k\n  OUTPUT: The deleted key-value pair\n\n  # use a connectBeforeAfter(p) function, that connect positions before and after p directly\n\n  p = SkipSearch(k)\n  v = value(p)\n  i = -1\n  while i < h:\n    i = i+1\n    connectBeforeAfter(p)\n    p = above(p)\n  n = n-1\n  return k, v\n\n\n\n\n'

In [28]:
# R-10.25
import random
def pop(self):
  k = random.randint(0, len(self))
  i = 0
  for e in self:
    if i == k:
      return self.discard(e) 
    else:
      i += 1


# 1. set is an unordered collection, it has elements, but not index. its pop() and discard() methods are working on elements rather than index


In [29]:
# R-10.26
def isdisjoint(self, other):
  s = set()
  for e in self:
    s.add(e)
  for e in other:
    s.add(e)
  if len(s) == len(self) + len(other):
    return True
  else:
    return False

def isdisjoint2(self, other):
  # a better method
  if len(self) < len(other):
    for e in self:
      if e in other:
        return False
  else:
    for e in other:
      if e in self:
        return False
  return True






# 1. use add() to add elements in a set, use insert() or append() to add elements in a list

In [30]:
# R-10.27
# sorted map: we need an order, and we need to find an item near a paticular key (the next one to celebrate a birthday)

In [31]:
# C-10.36
'''
a.
N is a prime number, so N is an odd
given i^2 % N = (N-i)^2 % N for i belongs to [1, N-1]
there are at most (N+1)/2 distinct values

b.
prime number N, N % 4 = 3
i belongs to [1, (N-1)/2]
i^2 % N = (N-i)^2 % N
-i^2 % N = -(N-i)^2 % N
when i = (N-1)/2, N-i = (N+1)/2, there are at most (N+1) distinct values


'''

'\na.\nN is a prime number, so N is an odd\ngiven i^2 % N = (N-i)^2 % N for i belongs to [1, N-1]\nthere are at most (N+1)/2 distinct values\n\nb.\nprime number N, N % 4 = 3\ni belongs to [1, (N-1)/2]\ni^2 % N = (N-i)^2 % N\n-i^2 % N = -(N-i)^2 % N\nwhen i = (N-1)/2, N-i = (N+1)/2, there are at most (N+1) distinct values\n\n\n'

In [32]:
# C-10.44
'''
in SkipSearch(k), we don't use above() and prev(), the output is the largest key that key(p) <= k, which is actually the prev() of the target
in SkipInsert(k, v), first determine the height of the new tower, once the next(p) > k, insert. and use below() to link the tower.
in SkipDel(k), once next(p) = k, connect p to next(next(p)), and below() from p



'''

"\nin SkipSearch(k), we don't use above() and prev(), the output is the largest key that key(p) <= k, which is actually the prev() of the target\nin SkipInsert(k, v), first determine the height of the new tower, once the next(p) > k, insert. and use below() to link the tower.\nin SkipDel(k), once next(p) = k, connect p to next(next(p)), and below() from p\n\n\n\n"